# Ensembl database table extraction

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from IPython.display import display, Markdown

import ensembl_genes

In [ ]:
# parameters cell
release = "104"

In [ ]:
ensg = ensembl_genes.Ensembl_Gene_Queries(release=release)
ensg.connection_url

In [ ]:
database = ensg.database
database

## All tables

Display the head of each table in the core database.
Together with the [schema documentation](https://uswest.ensembl.org/info/docs/api/core/core_schema.html), this helps understand what is in the Ensembl database.

In [ ]:
query = f'''
SHOW FULL TABLES IN {database}
'''
table_df = pd.read_sql(sql=query, con=ensg.connection_url)
table_df.head(2)

In [ ]:
# number of tables (note that there are no views)
table_df.Table_type.value_counts()

In [ ]:
tables = sorted(table_df[f"Tables_in_{database}"])
len(tables)

In [ ]:
for table in tables:
    display(Markdown(f"## {table}"))
    query = f"SELECT * FROM {table} LIMIT 5"
    df = pd.read_sql(sql=query, con=ensg.connection_url).convert_dtypes()
    display(df)

## Extract data

## gene attrib counts

In [ ]:
ensg.run_query("gene_attrib_counts").head(15)

## genes

In [ ]:
ensg.gene_df.head()

In [ ]:
# clone-based genes no longer get a symbol and are filled with the stable ID
# https://www.ensembl.info/2021/03/15/retirement-of-clone-based-gene-names/
ensg.gene_df.query("gene_symbol == ensembl_gene_id").head(2)

In [ ]:
# which external database the gene symbol derives from versus the ensembl source
pd.crosstab(
    ensg.gene_df.ensembl_source,
    ensg.gene_df.gene_symbol_source_db.fillna("missing (clone-based)"),
    margins=True,
)

In [ ]:
ensg.gene_df.coord_system.value_counts().head(10)

In [ ]:
ensg.gene_df.gene_biotype.value_counts().head(10)

In [ ]:
ensg.gene_df.seq_region_exc_type.value_counts(dropna=False)

In [ ]:
ensg.gene_df.mhc.value_counts()

In [ ]:
len(ensg.gene_df)

## alternative gene alleles

Related:

- [OTP: Origin of genes_with_non_reference_ensembl_ids.tsv](https://github.com/opentargets/platform/issues/702)
- [biostars: map between different assemblies of one ensembl release](https://www.biostars.org/p/143956/)
- using `attrib_type.code = "non_ref"` for `primary_assembly` doesn't appear to return any results

In [ ]:
ensg.alt_allele_df.head()

In [ ]:
# looks like non_ref isn't set for human genes
query = '''
SELECT *
FROM gene_attrib
LEFT JOIN attrib_type
  ON gene_attrib.attrib_type_id = attrib_type.attrib_type_id
WHERE attrib_type.code = "non_ref"
LIMIT 5
'''
pd.read_sql(sql=query, con=ensg.connection_url)

In [ ]:
ensg.alt_allele_df.alt_allele_attrib.value_counts()

In [ ]:
ensg.alt_allele_df.query("is_representative_gene").representative_gene_method.value_counts()

In [ ]:
ensg.gene_df.query("ensembl_gene_id != ensembl_representative_gene_id").head(2)

# replaced ID converter

A single `old_stable_id` can map to multiple `new_stable_id`. For example, `ENSG00000152006`

https://uswest.ensembl.org/Homo_sapiens/Tools/IDMapper/Results?tl=AzhM62SpkvdiLC4H-6808613

Requested ID | Matched ID(s) | Releases
-- | -- | --
ENSG00000152006 | ENSG00000196273 | 26: ENSG00000196273.1
ENSG00000152006 | ENSG00000197016 | 26: ENSG00000197016.1
ENSG00000152006 | ENSG00000196239 | 26: ENSG00000196239.1

In [ ]:
ensg.old_to_new_df.head(2)

In [ ]:
# some ensembl genes replaced by many new ensembl genes
ensg.old_to_new_df.old_ensembl_gene_id.value_counts().head(2)

In [ ]:
# example
ensg._update_ensembl_gene("ENSG00000152006")

In [ ]:
ensg.old_to_newest_df.head(2)

In [ ]:
len(ensg.old_to_newest_df)

In [ ]:
ensg.old_to_newest_df.is_current.value_counts()

## omni-updater

The omni-updater dataset is designed to convert ensembl gene IDs from input data to the current, representative ensembl_gene_ids for this ensembl release. It assumes:

- users want to update outdated genes with their replacements
- users want a dataset of representative genes only, and want to convert alternative alleles to representative genes

An inner join of a dataset with `update_df` on `input_ensembl_gene_id` will do the following:

- produce output ensembl_gene_ids that are current and representatives
- update outdated genes with their current identifiers. Outdated genes with no current replacement will be removed by the inner join.
- update alternative gene alleles with their representatives
- genes that are already represenative and current will map to themselves

In [ ]:
ensg.update_df.head(2)

In [ ]:
ensg.update_df.sort_values("input_maps_to_n_genes", ascending=False).head(2)

In [ ]:
ensg.update_df.sort_values("n_inputs_map_to_gene", ascending=False).head(2)

In [ ]:
(ensg.update_df.input_maps_to_n_genes == 1).mean()

In [ ]:
ensg.update_df.query("ensembl_gene_id == 'ENSG00000256263'")

In [ ]:
print(
    f"The omni-updater contains {len(ensg.update_df):,} rows for mapping "
    f"{ensg.update_df.input_ensembl_gene_id.nunique():,} input genes to "
    f"{ensg.update_df.ensembl_gene_id.nunique():,} current, representative genes."
)

In [ ]:
# https://useast.ensembl.org/Homo_sapiens/Tools/IDMapper/Results?tl=P45VLMbogubpI0QA-6815464
ensg.update_df.query("input_ensembl_gene_id == 'ENSG00000201456'").head(3)

## cross-refrences (xrefs)

In [ ]:
ensg.xref_df.head()

In [ ]:
# datasets where there are ensembl_gene_id-xref_source-xref_accession pairs might not be distinct 
xref_dup_df = ensg.xref_df[ensg.xref_df.duplicated(subset=["ensembl_gene_id", "xref_source", "xref_accession"], keep=False)]
xref_dup_df.xref_source.value_counts()

In [ ]:
# xref sources versus info_types
pd.crosstab(ensg.xref_df.xref_source, ensg.xref_df.xref_info_type, margins=True)

## Gene Ontology xrefs

In [ ]:
ensg.xref_go_df.head(3)

In [ ]:
# GO terms for CCR5
# compare to http://useast.ensembl.org/Homo_sapiens/Gene/Ontologies/molecular_function?g=ENSG00000160791
sorted(ensg.xref_go_df.query("ensembl_gene_id == 'ENSG00000160791'").go_label)

## lrg xrefs

In [ ]:
ensg.xref_lrg_df.head(2)

In [ ]:
len(ensg.xref_lrg_df)

### ncbigene xrefs

In [ ]:
ensg.xref_ncbigene_df.head()

In [ ]:
# ensembl gene mapped to by multiple ncbigenes
ensg.xref_ncbigene_df.ensembl_representative_gene_id.value_counts().head(3)

In [ ]:
len(ensg.xref_ncbigene_df), ensg.xref_ncbigene_df.ensembl_representative_gene_id.duplicated().sum()

In [ ]:
# ncbigene mapped to by multiple ensembl genes, likely due to alt gene alleles
ensg.xref_ncbigene_df.ncbigene_id.value_counts().head(3)

In [ ]:
len(ensg.xref_ncbigene_df), ensg.xref_ncbigene_df.ncbigene_id.duplicated().sum()

In [ ]:
# ensg.xref_ncbigene_df.query("ensembl_representative_gene_id == 'ENSG00000231500'")
# ensg.xref_ncbigene_df.query("ncbigene_id == '51206'")

In [ ]:
repr_ensembl_gene_ids = set(ensg.gene_df.ensembl_representative_gene_id)
len(repr_ensembl_gene_ids)

In [ ]:
# many of these genes should probably be alternative alleles rather than representative
ensg.gene_df.query("not primary_assembly and ensembl_gene_id==ensembl_representative_gene_id")